In [0]:
!pip3 install torchvision

In [0]:
# Understanding gradient descent

x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

w = 1.0

def forward(x):
    return x*w

def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y)*(y_pred - y)

def gradient(x, y):
    return 2*x*(x*w - y)

print('My prediction before training', 4, forward(4))

for epoch in range(10):
    for x_val, y_val in zip(x_data, y_data):
        grad = gradient(x_val, y_val)
        learning_rate = 0.01
        w = w - learning_rate * grad
        print('\tgrad:', x_val, y_val, round(grad, 2))
        l = loss(x_val, y_val)
    print('Progress:', epoch, "w=", round(w,2), 'loss', round(l, 2))

print('My new prediction after training', forward(4))

In [0]:
# Now lets apply PyTorch

import torch
from torch.autograd import Variable

x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

w = Variable(torch.Tensor([1.0]), requires_grad = True)

# As we can use functions from previous cell, i`m not gonna to rewrite it

print('My prediction before training {} : {} '.format(4, forward(4).data[0]))

for epoch in range(10):
    for x_val, y_val in zip(x_data, y_data):
        l = loss(x_val, y_val)
        l.backward()
        print('\tgrad:', x_val, y_val, w.grad.data[0])
        learning_rate = 0.01
        w.data = w.data - learning_rate * w.grad.data
        
        w.grad.data.zero_()
        
    print('Progress: {} loss: {}'.format(epoch, l.data[0]))
    
print('My new prediction after training', forward(4).data[0])

In [0]:
# Linear regression in PyTorch

import torch
from torch.autograd import Variable

x_data = Variable(torch.Tensor([[1.0], [2.0], [3.0]]))
y_data = Variable(torch.Tensor([[2.0], [4.0], [6.0]]))

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.linear = torch.nn.Linear(1, 1)
        
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred
    
model = Model()

criterion = torch.nn.MSELoss(size_average = False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(100):
    
    y_pred = model(x_data)
    
    loss = criterion(y_pred, y_data)
    print(epoch, loss.data[0])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
output = Variable(torch.Tensor([[4.0]]))
y_pred = model(output)
print("prediction after the training", 4, model(output).data[0][0])

In [9]:
# Deep Neural Network example

# Downloaded this dataset from kaggle. Just type diabetes.csv to find that one
d = 'diabetes.csv'

import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F

xy = np.loadtxt(d, delimiter=',', dtype=np.float32, skiprows=1)

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]],
      dtype=float32)

In [0]:
x_data = Variable(torch.from_numpy(xy[:, 0:-1]))
y_data = Variable(torch.from_numpy(xy[:, [-1]]))

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.l1 = torch.nn.Linear(8, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1)
        
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        
        out1 = F.relu(self.l1(x))
        out2 = F.relu(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        
        return y_pred
    
model = Model()

criterion = torch.nn.BCELoss(size_average = True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    y_pred = model(x_data)
    
    loss = criterion(y_pred, y_data)
    
    print('Epoch: {}, loss: {}'.format(epoch, loss.data[0]))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    

In [17]:
# CNN in Pytorch
# Ahh, again MNIST dataset

import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

batch_size = 128

train_dataset = datasets.MNIST(root='/data/', train=True, transform=transforms.ToTensor(), download = True)
test_dataset = datasets.MNIST(root='/data/', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        
        self.fc = nn.Linear(320, 10)
        
    def forward(self, x):
        initial_size = x.size(0)
        
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(initial_size, -1)
        x = self.fc(x)
        
        return F.log_softmax(x)
    
model = CNN()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{}] ({:.0f}%]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data), len(train_loader.dataset), 
                                                                          100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy:{}/{} ({:0f}%\n)'.format(test_loss, correct, len(test_loader.dataset), 
                                                                              100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 3):
    train(epoch)
    test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/60000] (0%]	Loss: 2.305419
Train Epoch: 1 [1280/60000] (2%]	Loss: 2.305606
Train Epoch: 1 [2560/60000] (4%]	Loss: 2.295037
Train Epoch: 1 [3840/60000] (6%]	Loss: 2.288960
Train Epoch: 1 [5120/60000] (9%]	Loss: 2.266279
Train Epoch: 1 [6400/60000] (11%]	Loss: 2.264006
Train Epoch: 1 [7680/60000] (13%]	Loss: 2.245130
Train Epoch: 1 [8960/60000] (15%]	Loss: 2.227899
Train Epoch: 1 [10240/60000] (17%]	Loss: 2.200622
Train Epoch: 1 [11520/60000] (19%]	Loss: 2.168125
Train Epoch: 1 [12800/60000] (21%]	Loss: 2.106326
Train Epoch: 1 [14080/60000] (23%]	Loss: 2.020199
Train Epoch: 1 [15360/60000] (26%]	Loss: 1.881897
Train Epoch: 1 [16640/60000] (28%]	Loss: 1.709609
Train Epoch: 1 [17920/60000] (30%]	Loss: 1.489804
Train Epoch: 1 [19200/60000] (32%]	Loss: 1.229862
Train Epoch: 1 [20480/60000] (34%]	Loss: 1.033832
Train Epoch: 1 [21760/60000] (36%]	Loss: 0.772351
Train Epoch: 1 [23040/60000] (38%]	Loss: 0.708301
Train Epoch: 1 [24320/60000] (41%]	Loss: 0.682998
Train Epoch: 1 [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.2653, Accuracy:9238/10000 (92.000000%
)
Train Epoch: 2 [0/60000] (0%]	Loss: 0.456573
Train Epoch: 2 [1280/60000] (2%]	Loss: 0.209446
Train Epoch: 2 [2560/60000] (4%]	Loss: 0.232900
Train Epoch: 2 [3840/60000] (6%]	Loss: 0.205544
Train Epoch: 2 [5120/60000] (9%]	Loss: 0.336632
Train Epoch: 2 [6400/60000] (11%]	Loss: 0.278762
Train Epoch: 2 [7680/60000] (13%]	Loss: 0.223898
Train Epoch: 2 [8960/60000] (15%]	Loss: 0.257477
Train Epoch: 2 [10240/60000] (17%]	Loss: 0.233646
Train Epoch: 2 [11520/60000] (19%]	Loss: 0.341477
Train Epoch: 2 [12800/60000] (21%]	Loss: 0.195001
Train Epoch: 2 [14080/60000] (23%]	Loss: 0.369264
Train Epoch: 2 [15360/60000] (26%]	Loss: 0.274097
Train Epoch: 2 [16640/60000] (28%]	Loss: 0.308343
Train Epoch: 2 [17920/60000] (30%]	Loss: 0.276862
Train Epoch: 2 [19200/60000] (32%]	Loss: 0.384606
Train Epoch: 2 [20480/60000] (34%]	Loss: 0.227146
Train Epoch: 2 [21760/60000] (36%]	Loss: 0.179087
Train Epoch: 2 [23040/60000] (38%]	Loss: 0.188329